In [179]:
#%pip install nltk
#%pip install pandas
#%pip install numpy
#%pip install scikit-learn
#%pip install spacy
#%pip install pymorphy2
#%pip install gensim

In [180]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import spacy
import sklearn
import pymorphy2
from pymorphy2 import MorphAnalyzer
import string
import re
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
import gensim

In [181]:
#define
nltk.download('stopwords')
nltk.download('punkt')
rus = "russian"
sw = stopwords.words(rus)
morph = MorphAnalyzer()
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
lancaster = LancasterStemmer()
snowball = SnowballStemmer(language=rus)
vectorizer = CountVectorizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NitghtWay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NitghtWay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [182]:
test_ds1_path = "C:\\course_work_git\\feature_engineere_NLP\\datasets\\test_ds1\\train.jsonl"

In [183]:
data = pd.read_json(test_ds1_path, lines=True)
data

,premise,label,hypothesis,verb,negation,genre,idx,no_negation
0,Сумма ущерба составила одну тысячу рублей. Уто...,entailment,Ранее местный житель совершал подобное правона...,судить,no_negation,kp,0,NaN
1,"Перебрасываясь словечками, они скользят глазам...",contradiction,Они что-то понимают,смотреть,no_negation,fiction,1,NaN
2,"— Разве что, — сказала она, — мы хотим где-ниб...",neutral,"Это “ Таганская ”, а не “ Тульская ”.",понять,no_negation,fiction,2,NaN
3,"Зима, наконец, показала свой характер.",contradiction,У зимы есть свой характер.,показать,no_negation,kp,3,NaN
4,ГуманностьБогаподобнымсценариемнепредполагаетс...,contradiction,Это должно быть просто.,сказать,no_negation,fiction,4,NaN
...,...,...,...,...,...,...,...,...
433,Тверские спортсмены показали отличный результа...,entailment,У тверских спортсменов пять новых медалей.,суметь,no_negation,kp,433,NaN
434,"Просто я не хочу с ним жить и, извини, больше ...",contradiction,Этого было вполне достаточно.,уверять,no_negation,fiction,434,NaN
435,"Второй аргумент: все читаем и видим, что надви...",neutral,Надвигается вторая волна кризиса.,видеть,negation,interfax,435,NaN
436,"— Да, сын. Здравствуй. — Ты можешь говорить, н...",neutral,Сын не занят,говорить,no_negation,fiction,436,NaN


In [184]:
type(data)

pandas.core.frame.DataFrame

In [185]:
data = data.drop(['label', 'hypothesis', 'verb','negation','genre','idx','no_negation'], axis=1)

In [186]:
data.head()

,premise
0,Сумма ущерба составила одну тысячу рублей. Уто...
1,"Перебрасываясь словечками, они скользят глазам..."
2,"— Разве что, — сказала она, — мы хотим где-ниб..."
3,"Зима, наконец, показала свой характер."
4,ГуманностьБогаподобнымсценариемнепредполагаетс...


In [187]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   premise  438 non-null    object
dtypes: object(1)
memory usage: 3.5+ KB


In [188]:
data.shape

(438, 1)

Вспомогательные методы

In [189]:
def print_table(array):
    table_width = 5
    for i in range(0, len(array), table_width):
        row = array[i:i + table_width]
        formatted_row = " ".join(["{:<10}".format(str(elem)) for elem in row])
        print(formatted_row)

In [190]:
def remove_punctuation(text):
    return ''.join([ch for ch in text if ch not in 
    string.punctuation])

In [191]:
def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text]) 
    # метод для удаления чисел

In [192]:
def remove_patterns(text):
    text = re.sub(patterns, ' ', text)
    return text

In [193]:
def remove_notalpha(text):
    return ''.join(i if i.isalpha() else ' ' for i in text) # 

Стемминг с использованием SnowBall

In [194]:
example_text = data['premise'][4]
print(type(example_text))
print(example_text)

<class 'str'>
ГуманностьБогаподобнымсценариемнепредполагается. Но Его благость остается в неприкосновенности. Непросто жить в таком мире, но кто сказал, что это должно быть просто?


In [195]:
def tokenize_snowball(text):
    text = remove_notalpha(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = remove_patterns(text)
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw or len(w) >= 11:
                continue
            else:
                all_words.append(snowball.stem(w))
            
    return all_words

In [196]:
print_table(tokenize_snowball(example_text))

но         ег         благост    оста       непрост   
жит        так        мир        сказа      эт        
должн      прост     


In [197]:
lancaster = LancasterStemmer()
def tokenize_lancaster(text):
    text = remove_notalpha(text)
    text = remove_numbers(text)
    text = remove_patterns(text)
    text = remove_punctuation(text)
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw or len(w) >= 11:
                continue
            else:
                all_words.append(lancaster.stem(w))
    return all_words

In [198]:
print_table(tokenize_lancaster(example_text))

но         его        благость   остается   непросто  
жить       таком      мире       сказал     это       
должно     просто    


Лемматизация с помощью PyMorphy2

In [199]:
def tokenize_morphy(text):
    text = remove_notalpha(text)
    text = remove_patterns(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw or len(w) >= 11:
                continue
            else:
                all_words.append(morph.normal_forms(w)[0])
    if len(all_words) > 2:
        #all_words = pd.Series(all_words)
        return all_words
    else:
        return None
    

In [200]:
print_table(tokenize_morphy(example_text))

но         он         благость   оставаться непросто  
жить       такой      мир        сказать    это       
должный    просто    


In [201]:
df_morphy = pd.DataFrame({'text':[tokenize_morphy(sample) for sample in data['premise']]})
df_snowball = pd.DataFrame({'text':[tokenize_snowball(sample) for sample in data['premise']]})
df_lancaster = pd.DataFrame({'text':[tokenize_lancaster(sample) for sample in data['premise']]})
df_lancaster = df_lancaster.dropna()
df_morphy = df_morphy.dropna()
df_snowball = df_snowball.dropna()

In [202]:
df_morphy.head()

,text
0,"[сумма, ущерб, составить, один, тысяча, рубль,..."
1,"[словечко, скользить, глаз, мой, город, как, с..."
2,"[разве, сказать, хотеть, выпить, кофе, я, каза..."
3,"[зима, показать, свой, характер]"
4,"[но, он, благость, оставаться, непросто, жить,..."


In [203]:
df_snowball.head()

,text
0,"[сумм, ущерб, состав, одн, тысяч, рубл, уточня..."
1,"[словечк, скольз, глаз, мо, город, как, смеют,..."
2,"[разв, сказа, хот, вып, коф, мне, кажет, сказа..."
3,"[зим, показа, сво, характер]"
4,"[но, ег, благост, оста, непрост, жит, так, мир..."


In [204]:
df_lancaster.head()

,text
0,"[сумма, ущерба, составила, одну, тысячу, рубле..."
1,"[словечками, скользят, глазами, моему, городу,..."
2,"[разве, сказала, хотим, выпить, кофе, мне, каж..."
3,"[зима, показала, свой, характер]"
4,"[но, его, благость, остается, непросто, жить, ..."


Векторизация текста с использованием словаря

In [205]:
def vectorize(doc):
    features = defaultdict(int)
    for token in doc:
        features[token] += 1
    return features

In [206]:
def make_dictionary_words(df):
    dictionary_whole_words = defaultdict(int)
    for doc in df['text']:
        for token in doc:
            dictionary_whole_words[token] += 1
    return dictionary_whole_words

In [207]:
def sentence_to_vector(sentence, dictionary_whole_words):
    sentence = np.array(sentence)
    vector = np.zeros(len(dictionary_whole_words))
    dict_values = np.array(list(dictionary_whole_words.values()))
    dict_keys = np.array(list(dictionary_whole_words.keys()))
    sorter = np.argsort(dict_keys)
    for idx in sorter[np.searchsorted(dict_keys, sentence, sorter = sorter)]:
        vector[idx] = dict_values[idx]
    return vector 

In [208]:
whole_dict = make_dictionary_words(df_morphy)
whole_dict

defaultdict(int,
            {'сумма': 4,
             'ущерб': 2,
             'составить': 3,
             'один': 22,
             'тысяча': 4,
             'рубль': 4,
             'уточняться': 3,
             'место': 16,
             'выехать': 3,
             'группа': 5,
             'который': 34,
             'установить': 8,
             'личность': 2,
             'они': 14,
             'оказаться': 14,
             'местный': 4,
             'житель': 7,
             'ранее': 7,
             'судить': 9,
             'подобный': 3,
             'словечко': 2,
             'скользить': 2,
             'глаз': 8,
             'мой': 25,
             'город': 6,
             'как': 26,
             'сметь': 5,
             'смотреть': 8,
             'понимать': 40,
             'разве': 3,
             'сказать': 122,
             'хотеть': 28,
             'выпить': 2,
             'кофе': 1,
             'я': 81,
             'казаться': 13,
             'на': 8,
       

In [222]:
print(sentence_to_vector(df_morphy['text'][1], whole_dict)[20:])

[2. 2. 8. ... 0. 0. 0.]


In [210]:
a = defaultdict(int)
a["aboba"] += 1
a["g"] += 1
a["ggg"] += 1
B = np.array(list(a.keys()))
print(B)

['aboba' 'g' 'ggg']


In [ ]:
for doc in df_snowball['text']:
    print(vectorize(doc))

In [ ]:
for doc in df_lancaster['text']:
    print(vectorize(doc))

Векторизация текста sklearn

In [213]:
corpus = data['premise']

In [ ]:
vectors = vectorizer.fit_transform(corpus)
print(vectors)

Векторизация текста Gensim

In [215]:
corpus = [doc for doc in df_morphy['text']]
id2word = gensim.corpora.Dictionary(corpus)
vectors = [id2word.doc2bow(doc) for doc in corpus]

Реализация Bag of words

In [1]:
print("lalalal")

lalalal
